<a href="https://colab.research.google.com/github/Guillermo-rv/Deep-Learning/blob/main/Transformers_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets transformers tensorflow


In [2]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset





In [3]:
dataset = load_dataset("imdb")

train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(16)



In [5]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),  # 🔹 SOLUCIÓN PARA EL ERROR
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)




In [6]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=3
)


Epoch 1/3
1563/1563 [==============================] - 1898s 1s/step - loss: 0.0236 - accuracy: 0.9969 - val_loss: 5.0972 - val_accuracy: 0.5000
Epoch 2/3
1563/1563 [==============================] - 1866s 1s/step - loss: 0.0483 - accuracy: 0.9937 - val_loss: 4.3170 - val_accuracy: 0.5000
Epoch 3/3
1563/1563 [==============================] - 1892s 1s/step - loss: 0.0633 - accuracy: 0.9853 - val_loss: 4.0459 - val_accuracy: 0.5000


In [7]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Exactitud en el conjunto de prueba: {test_acc:.4f}")


1563/1563 [==============================] - 476s 305ms/step - loss: 4.0459 - accuracy: 0.5000
Exactitud en el conjunto de prueba: 0.5000


In [8]:
print("Ejemplos en entrenamiento:", len(train_texts))
print("Ejemplos en prueba:", len(test_texts))


Ejemplos en entrenamiento: 25000
Ejemplos en prueba: 25000


In [ ]:
# 🔹 Ajustar la tasa de aprendizaje y entrenar más tiempo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),  # 🔹 Aumentamos a 5e-5
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# 🔹 Entrenar con más épocas y batch más grande
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5,  # 🔹 Aumentamos de 3 a 5 épocas
    batch_size=32  # 🔹 Tamaño de batch más grande para mejor estabilidad
)


Epoch 1/5
